# Deploy and Run the SQD-PCM Function from the Cleveland Clinic Foundation

This interactive guide shows how to upload the SQD-PCM function template to Qiskit Serverless and run an example workload.

### Requirements

This guide was developed with the following package versions:

```
qiskit-ibm-catalog == 0.5.0
qiskit-ibm-runtime == 0.38.0
numpy
```

## 1. Autentication

Use `qiskit-ibm-catalog` to authenticate to `QiskitServerless` with your API key (token), which you can find on the [IBM Quantum Platform](https://quantum.cloud.ibm.com) dashboard. This will allow you to locally instantiate the serverless client to upload or run the selected function:

```python
from qiskit_ibm_catalog import QiskitServerless
serverless = QiskitServerless(token="MY_TOKEN")
```

You can optionally use `save_account()` to save your credentials in your local environment (see the [Set up your IBM Cloud account](/docs/guides/cloud-setup#cloud-save) guide). Note that this writes your credentials to the same file as [`QiskitRuntimeService.save_account()`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#save_account):

```python
QiskitServerless.save_account(token="MY_TOKEN")
```

If the account is saved, there is no need to provide the token to authenticate:

In [1]:
from qiskit_ibm_catalog import QiskitServerless

# Authenticate to the remote cluster
serverless = QiskitServerless(channel="ibm_quantum")

## 2. Upload the Function

To upload a Qiskit Function, you must first instantiate a `QiskitFunction` object that defines the function source code. The title will allow you to identify the function once it's in the remote cluster. The main entry point is the file that contains `if __name__ == "__main__"`. If your workflow requires additional source files, you can define a working directory that will be uploaded together with the entry point.

In [2]:
from qiskit_ibm_catalog import QiskitFunction

template = QiskitFunction(
    title="sqd_pcm_template",
    entrypoint="sqd_pcm.py",
    working_dir="./source_files/",  # all files in this directory will be uploaded
)
print(template)

QiskitFunction(sqd_pcm_template)


Once the instance is ready, upload it to serverless:

In [3]:
serverless.upload(template)

QiskitFunction(sqd_pcm_template)

To check if the program successfully uploaded, use `serverless.list()`:

In [4]:
serverless.list()

[QiskitFunction(template_hamiltonian_simulation),
 QiskitFunction(hamiltonian_simulation_template),
 QiskitFunction(sqd_pcm_template)]

## 3. Loand and Run the Function remotely


The function template has been uploaded, so you can run it remotely with Qiskit Serverless. First, load the template by name:

In [5]:
template = serverless.load("sqd_pcm_template")
print(template)

QiskitFunction(sqd_pcm_template)


Next, run the template with the domain-level inputs for Hamiltonian simulation. This example specifies a methanol-based workload.

In [6]:
molecule = {
    "atom": """
    O -0.04559 -0.75076 -0.00000;
    C -0.04844 0.65398 -0.00000;
    H 0.85330 -1.05128 -0.00000;
    H -1.08779 0.98076 -0.00000;
    H 0.44171 1.06337 0.88811;
    H 0.44171 1.06337 -0.88811
    """,  # Must be specified
    "basis": "cc-pvdz",  # default is "sto-3g"
    "spin": 0,  # default is 0
    "charge": 0,  # default is 0
    "verbosity": 0,  # default is 0
    "number_of_active_orb": 12,  # Must be specified
    "number_of_active_alpha_elec": 7,  # Must be specified
    "number_of_active_beta_elec": 7,  # Must be specified
    "avas_selection": ["%d O %s" % (k, x) for k in [0] for x in ["2s", "2px", "2py", "2pz"]]
    + ["%d C %s" % (k, x) for k in [1] for x in ["2s", "2px", "2py", "2pz"]]
    + ["%d H 1s" % k for k in [2, 3, 4, 5]],
    # default is None
    "datafiles_name": "methanol",
    # Must be specified. Name without the file extension.
    # This name will be assigned for checkpoint, fcidump, and count_dictionary files
    # as well as the name of the SQD results folder containing the
    # "bitstring_matrix" and "sci_vector" for each batch.
}

solvent_options = {
    "method": "IEF-PCM",  # other available methods are COSMO, C-PCM, SS(V)PE, see https://manual.q-chem.com/5.4/topic_pcm-em.html
    "eps": 78.3553,  # value for water
}

lucj_options = {
    "initial_layout": [
        0,
        14,
        18,
        19,
        20,
        33,
        39,
        40,
        41,
        53,
        60,
        61,
        2,
        3,
        4,
        15,
        22,
        23,
        24,
        34,
        43,
        44,
        45,
        54,
    ],
    "dynamical_decoupling_choice": True,
    "twirling_choice": True,
    "number_of_shots": 200000,
    "optimization_level": 3,
}

sqd_options = {
    "sqd_iterations": 3,
    "number_of_batches": 10,
    "samples_per_batch": 1000,
    "max_davidson_cycles": 200,
}

backend_name = "test_eagle_eu-de"

In [7]:
job = template.run(
    backend_name=backend_name,
    molecule=molecule,
    solvent_options=solvent_options,
    lucj_options=lucj_options,
    sqd_options=sqd_options,
)
print(job.job_id)

9385080f-2932-45b4-b64e-abd9f71c9824


Check the status of the job:

In [8]:
job.status()

'QUEUED'

After the job is running, you can fetch logs created from the `logger.info` outputs. These can provide actionable information about the progress of the Hamiltonian simulation workflow. For example, the value of the objective function during the iterative component of AQC, or the two-qubit depth of the final ISA circuit intended for execution on hardware.

In [9]:
print(job.logs())

No logs yet.


Block the rest of the program until a result is available. After the job is done, you can retrieve the results. These include the domain-level output of Hamiltonian simulation (expectation value) and useful metadata.

In [10]:
result = job.result()

result

QiskitServerlessException: {}

After the job completes, the entire logging output will be available.

In [11]:
print(job.logs())

2025-05-05 08:39:24,466	INFO job_manager.py:531 -- Runtime env is setting up.
Traceback (most recent call last):
  File "/tmp/ray/session_2025-05-05_08-39-05_217882_1/runtime_resources/working_dir_files/_ray_pkg_8258c564a20f147b/sqd_pcm.py", line 25, in <module>
    import ffsim
ModuleNotFoundError: No module named 'ffsim'

